# All charts
---

## The data sets we used are:
* City of Austin, austin_311_service_requests.csv
* The Travis county zip codes from the US Census Bureau.
* Social/Economic census data for Austin, TX by zipcode from the US Census Bureau
* City of Austin Household statistics from the US Census Bureau.

### Number of 311 calls by month

### Number of 311 calls by type

### breakdown of call categories if we have that information

### locations of callers (bubble plot?)

### Number of calls per capita by zipcode

### Number of calls by median household income of zip

### Number of calls by crime rate of zip

### Percent of call types by median household income (stacked bar)

### Number of calls by Temperature scatter

### Number fo calls by weekday